In [34]:
# Parte 1 classificação e análise CONASEMS - listar os de termos de filtragem em cada um dos trabalhos selecionados

import pandas as pd
import re

# Listas de termos que serão utilizados como filtros (termos relacionados à tecnologia)
patterns = [
    r'4G', r'\b4G\b', r'\(4G\)', r'5G', r'\b5G\b', r'\(5G\)', r'\bAI\b', r'\(AI\)', r'\bIA\b', r'\(IA\)', 
    r'aplicativo', r'\bAPI\b', r'\(API\)', r'aprendizado de máquina', r'aprendizagem profunda', r'assistente virtual', 
    r'augmented reality', r'automação', r'big data', r'Bluetooth', r'blockchain', r'celular', r'celulares', r'chatbot', 
    r'chatgpt', r'cloud', r'computação cognitiva', r'computação ubíqua', r'conectividade', r'curso autoinstrucional', 
    r'cursos massivos online', r'dashboard', r'dados massivos', r'data lake', r'data mining', r'datacenter', r'deep learning', 
    r'device', r'dispositivo portátil', r'\bEAD\b', r'\(EAD\)', r'e-learning', r'e-saúde', r'ehealth', r'ensino à distância', 
    r'\bERP\b', r'\(ERP\)', r'Enterprise Resource Planning', r'\bESP32\b', r'\(ESP32\)', r'facebook', r'gamificação', 
    r'geoprocessamento', r'georreferenciamento', r'geotecnologia', r'grande modelo de linguagem', r'grandes modelos de linguagem', 
    r'instagram', r'interface digital', r'internet', r'internet das coisas', r'interoperabilidade', r'intervenção digital', 
    r'inteligente', r'inteligência artificial', r'inteligência aumentada', r'\bIoT\b', r'\(IoT\)', r'large language model', 
    r'\bllm\b', r'\(llm\)', r'machine learning', r'mhealth', r'mídia social', r'mídias sociais', r'mixed reality', 
    r'mobile', r'MOOC', r'\bMicrocontrolador\b', r'\(Microcontrolador\)', r'nuvem', r'painel de indicador', 
    r'painel de indicadores', r'painéis de indicador', r'painéis de indicadores', r'planilha', r'plataforma', r'PowerBI', 
    r'PowerPoint', r'prontuário eletrônico', r'privacidade de dados', r'proteção de dados', r'raspberry', 
    r'realidade aumentada', r'realidades aumentadas', r'realidade mista', r'realidades mistas', r'realidade virtual', 
    r'realidades virtuais', r'rede móvel', r'rede neural', r'rede social', r'redes sociais', r'resolução digital', 
    r'robótica', r'\bSIG\b', r'\(SIG\)', r'saúde digital', r'saúde móvel', r'segurança digital', r'sensor', r'sensores', 
    r'sistema de gestão de saúde', r'sistemas de gestão de saúde', r'sistema de informação', r'sistemas de informação', 
    r'sistema de informações', r'sistemas de informações', r'sistema digital', r'sistemas digitais', r'site', r'smartphone', 
    r'smartphones', r'solução digital', r'soluções digitais', r'software', r'tecnologia', r'tecnologias', r'teleassistência', 
    r'teleatendimento', r'telecirurgia', r'teleconsulta', r'teleconsultoria', r'telediagnóstico', r'teleeducação', 
    r'tele-espirometria', r'telefônica', r'telefone', r'telegram', r'telemedicina', r'telemonitoramento', r'teleorientação', 
    r'telepsicologia', r'telerradiologia', r'telespirometria', r'telessaúde', r'tele-saúde', r'teletriagem', 
    r'televigilância', r'transformação digital', r'transformações digitais', r'vídeogame', r'vídeogames', r'video', 
    r'videos', r'virtual', r'web', r'website', r'Whats', r'Watts', r'Whatts', 
    r'\bPEC\b', r'\(PEC\)', r'\bPEC-SUS\b', r'\(PEC-SUS\)', r'\bE-SUS\b', r'\(E-SUS\)', r'\besus\b', r'\(esus\)'
]

# Compilar padrões de regex para eficiência
compiled_patterns = [re.compile(pat, re.IGNORECASE) for pat in patterns]

# Função para limpar os padrões removendo \b, parênteses e barras invertidas
def clean_pattern(pattern):
    return re.sub(r'\\b|\\|\(|\)', '', pattern)

# Função para identificar e listar padrões únicos encontrados em cada resumo
def extract_unique_patterns(text):
    matches = set()  # Usar set para garantir unicidade
    
    # Usar re.sub com uma função lambda para capturar padrões limpos
    for pattern in compiled_patterns:
        re.sub(pattern, lambda m: matches.add(clean_pattern(pattern.pattern)), text)
    
    return list(matches)  # Converte para lista antes de salvar

# Carregar o arquivo Excel
df_selecionadas = pd.read_excel("experiencias_selecionadas_CONASEMS.xlsx")

# Inicializar a coluna 'tipos_tecnologia' com listas vazias
df_selecionadas["tipos_tecnologia"] = [[] for _ in range(len(df_selecionadas))]

# Iterar sobre o DataFrame para extrair padrões únicos
for idx, linha in df_selecionadas.iterrows():
    resumo_completo = linha["resumo_completo"]
    
    # Obter lista de padrões únicos encontrados
    unique_patterns = extract_unique_patterns(resumo_completo)
    
    # Atualizar a coluna 'tipos_tecnologia' com os padrões encontrados
    df_selecionadas.at[idx, "tipos_tecnologia"] = unique_patterns

# Salvar o DataFrame atualizado
df_selecionadas.to_excel("experiencias_selecionadas_tipos_tecnologia_CONASEMS.xlsx", index=False)
df_selecionadas.to_csv("experiencias_selecionadas_tipos_tecnologia_CONASEMS.csv", index=False)
df_selecionadas.to_json("experiencias_selecionadas_tipos_tecnologia_CONASEMS.json", orient="records", lines=True)


In [46]:
# Parte 2 classificação e análise CONASEMS - contagem da ocorrência de trabalhos que contém os termos de busca e ordenamento dos 
# termos mais utilizados

import pandas as pd
from collections import Counter
import ast

# Carregar o arquivo CSV com os trabalhos e tipos de tecnologia
df_selecionadas = pd.read_csv("experiencias_selecionadas_tipos_tecnologia_CONASEMS.csv")

# Converter a coluna 'tipos_tecnologia' de string para lista, se necessário
df_selecionadas['tipos_tecnologia'] = df_selecionadas['tipos_tecnologia'].apply(ast.literal_eval)

# Carregar o arquivo de correlação entre termos e temas
df_correlacao = pd.read_excel("correlacao_termos_temas.xlsx")
df_correlacao = df_correlacao.sort_values(by='termo').reset_index(drop=True)

# Anos presentes no DataFrame
anos = [2018, 2019, 2021, 2022]

# Dicionários para armazenar contagens por ano
contagem_termos_por_ano = {ano: Counter() for ano in anos}
contagem_temas_por_ano = {ano: Counter() for ano in anos}
contagem_termos_total = Counter()

# Iterar pelos anos para calcular as frequências
for ano in anos:
    # Filtrar o DataFrame para o ano específico
    df_ano = df_selecionadas[df_selecionadas["ano"] == ano]
    
    # Unificar todas as listas de 'tipos_tecnologia' para o ano
    todos_tipos = sum(df_ano['tipos_tecnologia'], [])
    
    # Contar a frequência de cada valor único para o ano específico
    contagem_termos_por_ano[ano] = Counter(todos_tipos)
    
    # Atualizar o contador total
    contagem_termos_total.update(todos_tipos)
    
    # Adicionar o tema a cada termo e contar a frequência dos temas
    for termo, frequencia in contagem_termos_por_ano[ano].items():
        tema = df_correlacao[df_correlacao['termo'] == termo]['tema'].values[0] if termo in df_correlacao['termo'].values else 'Desconhecido'
        contagem_temas_por_ano[ano][tema] += frequencia

# Criar o DataFrame para Contagem de Termos com colunas para cada ano
df_contagem_termos = pd.DataFrame()
df_contagem_termos['Tipo de Tecnologia'] = df_correlacao['termo']
for ano in anos:
    df_contagem_termos[f'{ano}'] = df_contagem_termos['Tipo de Tecnologia'].apply(lambda termo: contagem_termos_por_ano[ano].get(termo, 0))
df_contagem_termos['Total'] = df_contagem_termos['Tipo de Tecnologia'].apply(lambda termo: contagem_termos_total.get(termo, 0))

# Criar o DataFrame para Contagem de Temas com colunas para cada ano
df_contagem_temas = pd.DataFrame()
df_contagem_temas['Tema'] = df_correlacao['tema'].unique()
for ano in anos:
    df_contagem_temas[f'{ano}'] = df_contagem_temas['Tema'].apply(lambda tema: contagem_temas_por_ano[ano].get(tema, 0))
df_contagem_temas['Total'] = df_contagem_temas[[f'{ano}' for ano in anos]].sum(axis=1)

# Criar a nova tabela com a lista de termos por tema e frequência total de cada tema e ano
lista_termos_por_tema = []
for tema in df_contagem_temas['Tema']:
    termos_por_ano = {}
    frequencias_por_ano = {}
    for ano in anos:
        # Obter termos e frequências e ordenar por frequência
        termos_ano = [f"{termo} ({contagem_termos_por_ano[ano][termo]})" for termo in df_correlacao[df_correlacao['tema'] == tema]['termo'] if contagem_termos_por_ano[ano].get(termo, 0) > 0]
        termos_ano_sorted = sorted(termos_ano, key=lambda x: int(x.split('(')[-1].strip(')')), reverse=True)
        termos_por_ano[f'Termos {ano}'] = ', '.join(termos_ano_sorted)
        frequencias_por_ano[f'{ano}'] = contagem_temas_por_ano[ano].get(tema, 0)
    
    # Calcular os termos totais (soma de todas as frequências) e ordenar
    termos_total = [f"{termo} ({contagem_termos_total[termo]})" for termo in df_correlacao[df_correlacao['tema'] == tema]['termo'] if contagem_termos_total.get(termo, 0) > 0]
    termos_total_sorted = ', '.join(sorted(termos_total, key=lambda x: int(x.split('(')[-1].strip(')')), reverse=True))
    frequencia_total = sum(frequencias_por_ano.values())
    
    lista_termos_por_tema.append({
        'Tema': tema,
        **termos_por_ano,
        'Termos Total': termos_total_sorted,
        **frequencias_por_ano,
        'Total': frequencia_total
    })

df_lista_termos_por_tema = pd.DataFrame(lista_termos_por_tema)

# Ordenar as tabelas conforme solicitado
df_contagem_termos = df_contagem_termos.sort_values(by='Total', ascending=False).reset_index(drop=True)
df_contagem_temas = df_contagem_temas.sort_values(by='Total', ascending=False).reset_index(drop=True)
df_lista_termos_por_tema = df_lista_termos_por_tema.sort_values(by='Total', ascending=False).reset_index(drop=True)

# Salvar em um arquivo Excel com as 4 planilhas:
# 1) Correlação de Termos e Temas
# 2) Contagem de Termos
# 3) Contagem de Temas
# 4) Lista de Termos por Tema e Frequência Total
with pd.ExcelWriter("contagem_termos_e_temas_por_ano_final.xlsx") as writer:
    df_correlacao.to_excel(writer, sheet_name="Correlação de Termos e Temas", index=False)
    df_contagem_termos.to_excel(writer, sheet_name="Contagem de Termos", index=False)
    df_contagem_temas.to_excel(writer, sheet_name="Contagem de Temas", index=False)
    df_lista_termos_por_tema.to_excel(writer, sheet_name="Lista de Termos por Tema", index=False)

# Exibir os DataFrames resultantes
print("Contagem de Termos:")
print(df_contagem_termos)
print("\nContagem de Temas:")
print(df_contagem_temas)
print("\nLista de Termos por Tema e Frequência Total:")
print(df_lista_termos_por_tema)



Contagem de Termos:
       Tipo de Tecnologia  2018  2019  2021  2022  Total
0              tecnologia     2    13    47    18     80
1              aplicativo     2    10    55    10     77
2                   Whats     0     2    54     9     65
3                telefone     4     1    51     5     61
4              plataforma     4     4    40     8     56
..                    ...   ...   ...   ...   ...    ...
60            inteligente     0     0     1     0      1
61                    SIG     1     0     0     0      1
62                  Watts     0     0     1     0      1
63             tele-saúde     0     0     0     1      1
64  transformação digital     0     0     0     1      1

[65 rows x 6 columns]

Contagem de Temas:
                                      Tema  2018  2019  2021  2022  Total
0  Tipos específicos de tecnologia digital    14    27   164    45    250
1                               Telessaúde     5    13   153    12    183
2                            Re

In [29]:
# Parte 3 classificação e análise CONASEMS - categorização dos dados por UF e região
import pandas as pd

# Carregar o JSON de cidades
df_cidades = pd.read_json("https://servicodados.ibge.gov.br/api/v1/localidades/municipios")

# Normalizar as colunas 'microrregiao' e 'regiao-imediata' para expandir seus dicionários em colunas separadas
df_cidades_mr = pd.json_normalize(df_cidades["microrregiao"])
df_cidades_ri = pd.json_normalize(df_cidades["regiao-imediata"])

# Remover as colunas originais e juntar as normalizadas com prefixos apropriados
df_cidades = df_cidades.drop(columns=["microrregiao", "regiao-imediata"]).join(
    df_cidades_mr.add_prefix("microrregiao_")
).join(
    df_cidades_ri.add_prefix("ri_")
)

# Carregar o JSON de unidades federativas (UF)
df_uf = pd.read_json("https://servicodados.ibge.gov.br/api/v1/localidades/estados")

# Normalizar a coluna 'regiao' para expandir seu dicionário em colunas separadas
df_uf_rg = pd.json_normalize(df_uf["regiao"])

# Remover a coluna original e juntar a normalizada com prefixo apropriado
df_uf = df_uf.drop(columns="regiao").join(df_uf_rg.add_prefix('regiao_'))

# Ajuste da coluna 'id' em df_cidades para evitar duplicação no merge
df_cidades = df_cidades.rename(columns={'id': 'id_cidade'})

# Carregar df_estudos
df_estudos = pd.read_json("experiencias_selecionadas_CONASEMS.json")

# Substituir "Brasilia" por "Brasília" na coluna 'cidade'
df_estudos['cidade'] = df_estudos['cidade'].replace("Brasilia", "Brasília")

# Corrigir o nome da cidade em df_estudos
#df_estudos['cidade'] = df_estudos['cidade'].replace("Brasilia", "Brasília")

# Adicionar a sigla do UF ao df_cidades para facilitar o merge
df_cidades = df_cidades.rename(columns={'microrregiao_mesorregiao.UF.sigla': 'sigla_uf', 
                                        'microrregiao_mesorregiao.UF.id': 'id_uf', 
                                        'ri_regiao-intermediaria.UF.regiao.sigla': 'regiao_sigla', 
                                        'ri_regiao-intermediaria.UF.regiao.nome': 'regiao_nome'})

# Fazer o merge usando a combinação de cidade e estado
df_estudos = df_estudos.merge(
    df_cidades[['nome', 'sigla_uf', 'id_cidade', 'id_uf', 'regiao_sigla', 'regiao_nome']],
    left_on=['cidade', 'estado'],
    right_on=['nome', 'sigla_uf'],
    how='left'
)

# Renomear as colunas conforme necessário
df_estudos = df_estudos.rename(columns={'estado': 'uf'})

# Remover colunas extras resultantes do merge
df_estudos = df_estudos.drop(columns=['nome', 'sigla_uf'])

# Salvar em JSON
df_estudos.to_json("experiencias_selecionadas_infos_geograficas_CONASEMS.json", index=False)

anos = sorted(df_estudos['ano'].unique())

# Tabela 1: UF, Região, Frequência por ano e Total
# Agrupar por UF, Região e Ano para contar a frequência de 'id'
df_uf_regiao = df_estudos.groupby(['uf', 'regiao_nome', 'ano']).size().unstack(fill_value=0).reset_index()

# Renomear colunas para que cada coluna do ano seja apenas o ano (sem prefixo)
df_uf_regiao.columns.name = None  # Remove o nome do índice da coluna
df_uf_regiao.columns = ['UF', 'Região'] + [str(ano) for ano in anos]

# Adicionar coluna 'Total' com a soma das frequências por linha
df_uf_regiao['Total'] = df_uf_regiao[[str(ano) for ano in anos]].sum(axis=1)

# Ordenar por Região e depois por UF de forma ascendente
df_uf_regiao = df_uf_regiao.sort_values(by=['Região', 'UF']).reset_index(drop=True)

# Exibir a primeira tabela
print("Tabela 1: UF, Região e Frequência por Ano")
print(df_uf_regiao)

# Tabela 2: Região e Frequência por ano e Total
# Agrupar por Região e Ano para contar a frequência de 'id'
df_regiao = df_estudos.groupby(['regiao_nome', 'ano']).size().unstack(fill_value=0).reset_index()

# Renomear colunas para que cada coluna do ano seja apenas o ano
df_regiao.columns.name = None
df_regiao.columns = ['Região'] + [str(ano) for ano in anos]

# Adicionar coluna 'Total' com a soma das frequências por linha
df_regiao['Total'] = df_regiao[[str(ano) for ano in anos]].sum(axis=1)

# Ordenar por Região de forma ascendente
df_regiao = df_regiao.sort_values(by='Região').reset_index(drop=True)

# Exibir a segunda tabela
print("\nTabela 2: Região e Frequência por Ano")
print(df_regiao)

# Caso queira salvar ambas as tabelas em um arquivo Excel
with pd.ExcelWriter("tabelas_frequencia_por_ano.xlsx") as writer:
    df_uf_regiao.to_excel(writer, sheet_name="UF_Regiao_Frequencia", index=False)
    df_regiao.to_excel(writer, sheet_name="Regiao_Frequencia", index=False)


df_uf: ['id', 'sigla', 'nome', 'regiao_id', 'regiao_sigla', 'regiao_nome']
df_cidades: ['id_cidade', 'nome', 'microrregiao_id', 'microrregiao_nome', 'microrregiao_mesorregiao.id', 'microrregiao_mesorregiao.nome', 'id_uf', 'sigla_uf', 'microrregiao_mesorregiao.UF.nome', 'microrregiao_mesorregiao.UF.regiao.id', 'microrregiao_mesorregiao.UF.regiao.sigla', 'microrregiao_mesorregiao.UF.regiao.nome', 'ri_id', 'ri_nome', 'ri_regiao-intermediaria.id', 'ri_regiao-intermediaria.nome', 'ri_regiao-intermediaria.UF.id', 'ri_regiao-intermediaria.UF.sigla', 'ri_regiao-intermediaria.UF.nome', 'ri_regiao-intermediaria.UF.regiao.id', 'regiao_sigla', 'regiao_nome']
df_estudos: ['id', 'titulo', 'cidade', 'uf', 'ano', 'autoria', 'coautoria', 'resumo_completo', 'resumo_completo_html', 'introducao', 'objetivos', 'metodologia', 'resultados', 'conclusoes', 'palavras_chave', 'link_experiencia', 'id_cidade', 'id_uf', 'regiao_sigla', 'regiao_nome']


In [33]:
# Parte 4 classificação e análise CONASEMS - proporção de estudos sobre tecnologia digital em saúde por ano

import pandas as pd
df_geral = pd.read_json("experiencias_completas_CONASEMS.json")
df_selecionado = pd.read_json("experiencias_selecionadas_CONASEMS.json")

df_geral = df_geral.rename(columns={"ano": "Ano"})
df_selecionado = df_selecionado.rename(columns={"ano": "Ano"})

df_ano_geral = df_geral[["Ano", "id"]].groupby("Ano").count().rename(columns={"id": "Todos os estudos"})
df_ano_selecionado = df_selecionado[["Ano", "id"]].groupby("Ano").count().rename(columns={"id": "Estudos selecionados"})

# Junta os DataFrames com base na coluna 'ano'
df_combinado = pd.merge(df_ano_geral, df_ano_selecionado, on="Ano", how="outer")


# Calcula a coluna '%' com a proporção arredondada para 2 decimais
df_combinado['Percentual de selecionados'] = (df_combinado['Estudos selecionados'] / df_combinado['Todos os estudos']).round(3)

# Mostra o resultado
# df_combinado.to_excel("proporcao_estudos_selecionados_ano.xlsx", index=False)

df_combinado

,Todos os estudos,Estudos selecionados,Percentual de selecionados
Ano,,,
2018,346,25,0.072
2019,500,32,0.064
2021,630,167,0.265
2022,337,47,0.139


In [44]:
# Parte 5 - gerar mapa de publicações por ano e por UF

import requests
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

# Carregar dados de estados com id e sigla
def carregar_dados_estados():
    url = "https://servicodados.ibge.gov.br/api/v1/localidades/estados?orderBy=nome"
    response = requests.get(url)
    estados = pd.DataFrame(response.json())
    return estados[['id', 'sigla']]

# Carregar malha de cada UF com a API do IBGE
def baixar_malhas_uf(estados):
    geodf = gpd.GeoDataFrame()
    for _, row in estados.iterrows():
        uf_id = row['id']
        uf_sigla = row['sigla']
        url = f"https://servicodados.ibge.gov.br/api/v3/malhas/estados/{uf_id}?formato=application/vnd.geo+json"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            gdf = gpd.GeoDataFrame.from_features(data["features"])
            gdf['sigla'] = uf_sigla
            gdf['id'] = uf_id
            geodf = pd.concat([geodf, gdf], ignore_index=True)
        else:
            print(f"Erro ao baixar a malha para {uf_sigla}")
    return geodf

# Função para gerar mapa de calor para um ano específico ou total
def gerar_mapa_calor(geodf, df_publicacoes, estados, ano=None):
    # Filtrar publicações por ano ou total
    if ano:
        publicacoes_por_uf = df_publicacoes[df_publicacoes['ano'] == ano].groupby('id_uf').size().reset_index(name='contagem_publicacoes')
        titulo = ano
        nome_arquivo = f"mapa_calor_uf_{ano}.png"
        bins = [0, 1, 5, 10, 15, 20]
        labels = ['Sem Publicações', '1-5', '6-10', '11-15', '16-20']
    else:
        publicacoes_por_uf = df_publicacoes.groupby('id_uf').size().reset_index(name='contagem_publicacoes')
        titulo = "Total"
        nome_arquivo = "mapa_calor_uf_total.png"
        bins = [0, 1, 5, 10, 15, 20, 30, np.inf]
        labels = ['Sem Publicações', '1-5', '6-10', '11-15', '16-20', '20-30', '>30']
    
    # Merge com GeoDataFrame usando a coluna 'id' para associar as UFs
    geodf = geodf.merge(publicacoes_por_uf, left_on="id", right_on="id_uf", how="left").fillna(0)
    geodf['categoria_calor'] = pd.cut(geodf['contagem_publicacoes'], bins=bins, labels=labels, include_lowest=True)

    # Definir o colormap em escala de cinza
    colormap = {
        'Sem Publicações': '#FFFFFF',  # Branco
        '1-5': '#d9d9d9',
        '6-10': '#bfbfbf',
        '11-15': '#969696',
        '16-20': '#636363',
        '20-30': '#404040',
        '>30': '#252525'
    }
    
    # Plotar o mapa com base nos bins
    fig, ax = plt.subplots(1, 1, figsize=(12, 10))
    geodf.boundary.plot(ax=ax, linewidth=1, color="black")  # Linhas divisórias
    # Atualizar a linha para evitar o aviso
    geodf.plot(color=geodf['categoria_calor'].map(colormap), linewidth=0.8, ax=ax, edgecolor='black', legend=False)

    # Adicionar as siglas dos estados e título
    for x, y, label in zip(geodf.geometry.centroid.x, geodf.geometry.centroid.y, geodf['sigla']):
        ax.text(x, y, label, fontsize=8, ha='center', color='black')
    
    ax.set_title(titulo, fontsize=15)
    ax.axis('off')

    # Configurar legenda com borda preta
    legend_patches = [mpatches.Patch(facecolor=colormap[label], edgecolor='black', label=label) for label in labels]
    ax.legend(handles=legend_patches, title="Faixas de Publicações", loc='lower left', frameon=True)

    # Salvar o mapa
    plt.savefig(nome_arquivo, dpi=300, bbox_inches='tight')
    plt.close()

# Carregar dados e executar a geração dos mapas
estados = carregar_dados_estados()
geodf = baixar_malhas_uf(estados)
df_estudos = pd.read_json("experiencias_selecionadas_infos_geograficas_CONASEMS.json")

# Gerar mapas por ano e um mapa total
anos = [2018, 2019, 2021, 2022]
for ano in anos:
    gerar_mapa_calor(geodf.copy(), df_estudos, estados, ano)

# Gerar o mapa total
gerar_mapa_calor(geodf.copy(), df_estudos, estados)
